### Imports

In [1]:
import sys
sys.dont_write_bytecode = True


import torch
import numpy as np
import random
import os

def set_seeds(seed_value=42):
    """Sets seeds for reproducibility."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)

set_seeds(42) 

import json
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import warnings
import logging
from datetime import datetime
warnings.filterwarnings('ignore')

from model import get_model
from config import CFG
from dataset import get_dataset_class
from transform import get_transforms
from runner import run_baseline, run_lodo

torch.manual_seed(CFG["system"]["seed"])
np.random.seed(CFG["system"]["seed"])

device = CFG["system"]["device"]
print(f"Device: {device}")
print(f"PyTorch: {torch.__version__}")

DS = "PACS"
MODEL_NAME = "resnet18"

c:\Users\Fatim_Sproj\anaconda3\envs\bacp\lib\site-packages\torch\cuda\__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


Device: cuda
PyTorch: 2.5.1+cu121


### DataLoading

In [2]:
train_transform, test_transform = get_transforms(img_size=224, augment=False, use_imagenet_norm=False)

DatasetClass = get_dataset_class(DS)

ld = DatasetClass(
    data_root=CFG["datasets"][DS]["root"],
    transform=train_transform,
    batch_size=CFG["train"]["batch_size"]
)

print("\nData loaders ready!")


Data loaders ready!


### Logging

In [3]:
dataset_name = DS
base_dir = os.path.join(os.getcwd(), dataset_name)
subdirs = ["logs", "checkpoints", "plots"]

for sub in subdirs:
    os.makedirs(os.path.join(base_dir, sub), exist_ok=True)

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_file = os.path.join(base_dir, "logs", f"train_{timestamp}.log")

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()
    ]
)

logger = logging.getLogger(f"{dataset_name}_logger")

logger.info(f"Initialized experiment directories for {dataset_name}")
logger.info(f"Logs: {os.path.join(base_dir, 'logs')}")
logger.info(f"Checkpoints: {os.path.join(base_dir, 'checkpoints')}")
logger.info(f"Plots: {os.path.join(base_dir, 'plots')}")

2025-11-14 21:56:04,965 | INFO | Initialized experiment directories for PACS
2025-11-14 21:56:04,966 | INFO | Logs: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\resnet18_experiments\PACS\logs
2025-11-14 21:56:04,966 | INFO | Checkpoints: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\resnet18_experiments\PACS\checkpoints
2025-11-14 21:56:04,967 | INFO | Plots: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\resnet18_experiments\PACS\plots


### Setup

In [4]:
domains = CFG["datasets"][DS]["domains"]
loaders = {d: {"train": ld.get_dataloader(d, train=True), "val": ld.get_dataloader(d, train=False)} for d in domains}
ckpt_root = os.path.join(base_dir, "checkpoints")
log_dir = os.path.join(base_dir, "logs")
plots_dir = os.path.join(base_dir, "plots")
os.makedirs(ckpt_root, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)
os.makedirs(plots_dir, exist_ok=True)
model_factory = lambda cfg, dataset_key: get_model(cfg,dataset=DS)
optimizer_fn = lambda model: optim.AdamW(model.parameters(), lr=CFG["train"]["lr"], weight_decay=CFG["train"].get("weight_decay", 0.01))
device = CFG["system"]["device"]
epochs = CFG["train"]["epochs"]

{
  "lodo_results": {
    "art_painting": 0.8341463414634146,
    "cartoon": 0.7974413646055437,
    "photo": 0.9580838323353293,
    "sketch": 0.6017811704834606
  },
  "timestamp": "20251004_020611"
}

### Leave One Domain Out

In [5]:
lodo_results, lodo_mean, lodo_summary = run_lodo(
    model_fn=model_factory,
    CFG=CFG,
    logger=logger,
    dataset_key=DS,
    domains=domains,
    loaders=loaders,
    optimizer_fn=optimizer_fn,
    device=device,
    ckpt_root=ckpt_root,
    log_dir=log_dir,
    epochs=epochs
)

2025-11-14 21:20:56,266 | INFO | === LODO: Leaving out domain 'art_painting' ===



=== LODO: Leaving out domain 'art_painting' ===


Evaluating: 100%|██████████| 16/16 [00:07<00:00,  2.28it/s]
2025-11-14 21:21:30,208 | INFO | [art_painting] Epoch 1/5 | Train - Loss: 0.6163, Cls: 0.6076, GRQO: 0.0086, Acc: 0.7997 | Val - Loss: 0.9780, Cls: 0.9762, GRQO: 0.0018, Acc: 0.7144
2025-11-14 21:21:30,342 | INFO | [art_painting] New best val acc: 0.7144


[art_painting] Epoch 1/5 | Train - Loss: 0.6163, Cls: 0.6076, GRQO: 0.0086, Acc: 0.7997 | Val - Loss: 0.9780, Cls: 0.9762, GRQO: 0.0018, Acc: 0.7144
[art_painting] New best val acc: 0.7144


Evaluating: 100%|██████████| 16/16 [00:07<00:00,  2.18it/s]
2025-11-14 21:22:03,475 | INFO | [art_painting] Epoch 2/5 | Train - Loss: 0.1448, Cls: 0.1418, GRQO: 0.0030, Acc: 0.9494 | Val - Loss: 0.7025, Cls: 0.7015, GRQO: 0.0009, Acc: 0.7964
2025-11-14 21:22:03,575 | INFO | [art_painting] New best val acc: 0.7964


...

[sketch] Epoch 3/5 | Train - Loss: 0.0195, Cls: 0.0172, GRQO: 0.0023, Acc: 0.9969 | Val - Loss: 1.2426, Cls: 1.2417, GRQO: 0.0009, Acc: 0.6264


Evaluating: 100%|██████████| 31/31 [00:13<00:00,  2.22it/s]
2025-11-14 21:31:29,457 | INFO | [sketch] Epoch 4/5 | Train - Loss: 0.0063, Cls: 0.0042, GRQO: 0.0021, Acc: 0.9998 | Val - Loss: 1.1994, Cls: 1.1986, GRQO: 0.0008, Acc: 0.6684
2025-11-14 21:31:29,599 | INFO | [sketch] New best val acc: 0.6684


[sketch] Epoch 4/5 | Train - Loss: 0.0063, Cls: 0.0042, GRQO: 0.0021, Acc: 0.9998 | Val - Loss: 1.1994, Cls: 1.1986, GRQO: 0.0008, Acc: 0.6684
[sketch] New best val acc: 0.6684


Evaluating: 100%|██████████| 31/31 [00:12<00:00,  2.40it/s]
2025-11-14 21:32:06,407 | INFO | [sketch] Epoch 5/5 | Train - Loss: 0.0035, Cls: 0.0016, GRQO: 0.0019, Acc: 1.0000 | Val - Loss: 1.2603, Cls: 1.2596, GRQO: 0.0007, Acc: 0.6564
2025-11-14 21:32:06,407 | INFO | [sketch] Best Acc: 0.6684
2025-11-14 21:32:06,407 | INFO | ------------------------------------------------------------
2025-11-14 21:32:06,407 | INFO | LODO finished | Mean Acc: 0.7844 | Summary saved to c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\resnet18_experiments\PACS\logs\lodo_summary_20251114_213206.json


[sketch] Epoch 5/5 | Train - Loss: 0.0035, Cls: 0.0016, GRQO: 0.0019, Acc: 1.0000 | Val - Loss: 1.2603, Cls: 1.2596, GRQO: 0.0007, Acc: 0.6564
[sketch] Best Acc: 0.6684
------------------------------------------------------------
LODO finished | Mean Acc: 0.7844
Summary saved to c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\resnet18_experiments\PACS\logs\lodo_summary_20251114_213206.json


### Baseline

In [5]:
baseline_results, baseline_mean = run_baseline(
    model_name=MODEL_NAME,
    CFG=CFG,
    logger=logger,
    dataset_key=DS,
    domains=domains,
    loaders=loaders,
    optimizer_fn=optimizer_fn,
    device=device,
    epochs=CFG["train"]["epochs"]
)

2025-11-14 21:56:05,155 | INFO | Initializing ResNet baseline: resnet18


Initializing ResNet baseline: resnet18


2025-11-14 21:56:05,437 | INFO | === Baseline LODO: Leaving out domain 'art_painting' ===



=== Baseline LODO: Leaving out domain 'art_painting' ===


2025-11-14 21:56:36,902 | INFO | [art_painting] Epoch 1/5 | Train - Loss: 0.4853, Acc: 0.8441 | Val Acc: 0.7213


...

[sketch] Epoch 3/5 | Train - Loss: 0.0204, Acc: 0.9993 | Val Acc: 0.5785


2025-11-14 22:05:31,558 | INFO | [sketch] Epoch 4/5 | Train - Loss: 0.0081, Acc: 1.0000 | Val Acc: 0.6106


[sketch] Epoch 4/5 | Train - Loss: 0.0081, Acc: 1.0000 | Val Acc: 0.6106


2025-11-14 22:05:54,619 | INFO | [sketch] Epoch 5/5 | Train - Loss: 0.0048, Acc: 1.0000 | Val Acc: 0.6098
2025-11-14 22:05:54,619 | INFO | [sketch] Best Val Acc: 0.6210
2025-11-14 22:05:54,619 | INFO | ------------------------------------------------------------
2025-11-14 22:05:54,619 | INFO | Baseline LODO (resnet18) finished | Mean Acc: 0.7537


[sketch] Epoch 5/5 | Train - Loss: 0.0048, Acc: 1.0000 | Val Acc: 0.6098
[sketch] Best Val Acc: 0.6210
------------------------------------------------------------
Baseline LODO (resnet18) finished | Mean Acc: 0.7537
